<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/04_Named_Entity_Recognition_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install fuzzywuzzy
! pip install intervaltree

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=27dbbe1181fa3939b6bfb387c5832094ffcd78c5a16728ab2fdc641c12f570ed
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig, DistilBertForTokenClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
from collections import defaultdict
from fuzzywuzzy import fuzz
from intervaltree import Interval, IntervalTree
from collections import Counter
import json
from openai import OpenAI
import pandas as pd

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# Define nested_dict function
def nested_dict():
  return defaultdict(nested_dict)

# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

# Split CamelCase to s e p a r a t e d text
def split_camel_case(text):
    # Use regex to insert a space before uppercase letters preceded by lowercase letters
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

In [6]:
# Read in dictionary with definitions
definitions = read_pickle("short_answer_dict.pkl")
all_definitions = read_pickle("final_combined_dict.pkl")

## Checking existing NER capability

In [7]:
# Existing tags from DistilBERT
tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cpu


In [8]:
tag_tokenizer.is_fast

True

In [9]:
# Methods built from DistilBERT NER

def idx2string(text):
    # Split on whitespace or punctuation, punctuation is used for tokens
    words = re.findall(r'\b\w+\b|[^\s\w]', text)
    my_dict = {i: word for i, word in enumerate(words)}
    return my_dict

def tokenidx2words(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    word, index = sub_dict['word'], sub_dict['index']
    my_dict[index] = word
  return my_dict

def tokenidx2entity(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    entity, index = sub_dict['entity'], sub_dict['index']
    my_dict[index] = entity
  return my_dict

def idx2wordpos(text, idx2string):
  my_dict = {}
  current_pos = 0
  for idx, word in idx2string.items():
    start = text.index(word, current_pos)
    end = start + len(word)
    my_dict[idx] = (start, end)
    current_pos = end
  return my_dict

def tokenidx2entityword(example2wordidx, ner_results):
  # Create an IntervalTree
  tree = IntervalTree(Interval(start, end, key) for key, (start, end) in example2wordidx.items())

  my_dict = dict()
  for sub_dict in ner_results:
    tag_start, tag_end = sub_dict['start'], sub_dict['end']
    tag_entity = sub_dict['entity']
    token_idx = sub_dict['index']

    overlapping_intervals = tree[tag_start : tag_end] # Intervals which contain the entitry recognised
    for interval in overlapping_intervals:
      my_dict[token_idx] = {'entity' : tag_entity, 'word_idx': interval.data}

  return my_dict

def baseline_tags_list(tokens, baseline_tags):
  start_entities = list(len(tokens)*'O')
  for i in baseline_tags:
    start_entities[i['index']] = i['entity']
    assert len(start_entities) == len(tokens)
  return start_entities

In [10]:
example = "A Hankel matrix is a type of structured matrix where each ascending skew-diagonal from left to right is constant. It is characterized by its entries being functions of a single variable, forming a pattern where each element depends only on the sum of its indices. Hankel matrices are often used in signal processing and control theory for polynomial and sequence analysis."
print(f"{example}\n")
example2idx = idx2string(example)
print(f"word index: {example2idx}")

example2wordidx = idx2wordpos(example, example2idx)
print(f"word start-end index: {example2wordidx}")

ner_results = tags(example)
print(f"tags: {ner_results}")

tokenized = tag_tokenizer(example, padding=True, truncation=True)

input_ids = tokenized['input_ids']
print(f"input_ids: {len(input_ids)}, {input_ids}")

mapper_dict = tokenidx2entityword(example2wordidx, ner_results)
print(f"mapper_dict: {mapper_dict}")

baseline_tags = baseline_tags_list(input_ids, ner_results)
print(f"baseline_tags: {len(baseline_tags)}, {baseline_tags}")

attention_mask = tokenized['attention_mask']
print(f"attention_mask: {len(attention_mask)}, {attention_mask}")

A Hankel matrix is a type of structured matrix where each ascending skew-diagonal from left to right is constant. It is characterized by its entries being functions of a single variable, forming a pattern where each element depends only on the sum of its indices. Hankel matrices are often used in signal processing and control theory for polynomial and sequence analysis.

word index: {0: 'A', 1: 'Hankel', 2: 'matrix', 3: 'is', 4: 'a', 5: 'type', 6: 'of', 7: 'structured', 8: 'matrix', 9: 'where', 10: 'each', 11: 'ascending', 12: 'skew', 13: '-', 14: 'diagonal', 15: 'from', 16: 'left', 17: 'to', 18: 'right', 19: 'is', 20: 'constant', 21: '.', 22: 'It', 23: 'is', 24: 'characterized', 25: 'by', 26: 'its', 27: 'entries', 28: 'being', 29: 'functions', 30: 'of', 31: 'a', 32: 'single', 33: 'variable', 34: ',', 35: 'forming', 36: 'a', 37: 'pattern', 38: 'where', 39: 'each', 40: 'element', 41: 'depends', 42: 'only', 43: 'on', 44: 'the', 45: 'sum', 46: 'of', 47: 'its', 48: 'indices', 49: '.', 50: 

In [11]:
# Model requires entities to be cased to recognise them

## Encoding the definitions

In [12]:
# Dictionary with existing tags - these are to be updated to the new custom tags
dict_file_name = 'preliminary_ner_results.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    preliminary_ner_results = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")

  preliminary_ner_results = dict()

  for key, value in definitions.items():
    text_index = idx2string(value)
    tokenized = tag_tokenizer(value, padding=True, truncation=True)
    tokens = tokenized['input_ids']
    # tokens = tag_tokenizer(value).tokens()
    ner_results = tags(value)
    preliminary_ner_results[key] = {
        "text" : value,
        "text_index": text_index,
        "text_start_end_index": idx2wordpos(value, text_index),
        "tokens" : tokens,
        "baseline_tags" : baseline_tags_list(tokens, ner_results),
        "input_ids" : tag_tokenizer.encode(value),
        "attention_mask" : tokenized['attention_mask']
        }

  with open(dict_file_name, 'wb') as file:
    pickle.dump(preliminary_ner_results, file)

In [13]:
# Checks
for key in preliminary_ner_results.keys():
  if len(preliminary_ner_results[key]["input_ids"]) != len(preliminary_ner_results[key]["baseline_tags"]):
    print(key)

In [14]:
print(f"dictionary length: {len(preliminary_ner_results)}")

print(f"Pythagorean theorem:")
for k, v in preliminary_ner_results["Pythagorean theorem"].items():
  print(f"{k}:\n   {v}")

dictionary length: 5388
Pythagorean theorem:
text:
   The Pythagorean Theorem describes the relationship between the sides of a right triangle, stating that the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides. This principle is fundamental in geometry and is used to determine distances and measurements in various applications. It highlights the intrinsic connection between the angles and sides of a right triangle.
text_index:
   {0: 'The', 1: 'Pythagorean', 2: 'Theorem', 3: 'describes', 4: 'the', 5: 'relationship', 6: 'between', 7: 'the', 8: 'sides', 9: 'of', 10: 'a', 11: 'right', 12: 'triangle', 13: ',', 14: 'stating', 15: 'that', 16: 'the', 17: 'square', 18: 'of', 19: 'the', 20: 'length', 21: 'of', 22: 'the', 23: 'hypotenuse', 24: 'is', 25: 'equal', 26: 'to', 27: 'the', 28: 'sum', 29: 'of', 30: 'the', 31: 'squares', 32: 'of', 33: 'the', 34: 'lengths', 35: 'of', 36: 'the', 37: 'other', 38: 'two', 39: 'sides', 40: '.', 41:

In [15]:
# Creating an exceptions dictionary with prepositions, conjunctions to bypass in NER
exceptions = ['and', 'or', 'but', 'nor', 'for', 'so', 'yet', 'because', 'although', 'since', 'while', 'if', 'either', 'neither', 'both', 'not', 'only', 'also', 'about', 'above', 'across', 'after',
              'against', 'along', 'among', 'around', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'beyond', 'by', 'despite', 'down', 'during', 'except', 'for', 'from', 'in',
              'inside', 'into', 'like', 'means', 'near', 'of', 'off', 'on', 'onto', 'out', 'outside', 'over', 'past', 'since', 'through', 'throughout', 'till', 'to', 'toward', 'under', 'underneath', 'until',
              'up', 'upon', 'with', 'within', 'without']

exception_input_ids = [tag_tokenizer.encode(exception)[1] for exception in exceptions]

## Encoding Named Entity Recognition Dictionary

In [16]:
# Read in mapped dictionary with NER
all_classifications = read_pickle("all_classifications.pkl")

In [17]:
print(f"all_classifications dictionary length: {len(all_classifications)}")
random_sample = random.sample(list(all_classifications.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_classifications dictionary length: 21696
Path Trace:
 {'text_index': {0: 'Path', 1: 'Trace'}, 'tokens': ['[CLS]', 'Path', 'Trace', '[SEP]'], 'input_ids': [101, 17392, 22681, 102], 'ner_mapper': [17392, 22681], 'ner_tags': ['O', 'B-CALCULUS-AND-ANALYSIS', 'I-CALCULUS-AND-ANALYSIS', 'O']}
Mean square displacement:
 {'text_index': {0: 'Mean', 1: 'square', 2: 'displacement'}, 'tokens': ['[CLS]', 'Mean', 'square', 'displacement', '[SEP]'], 'input_ids': [101, 25030, 1961, 15167, 102], 'ner_mapper': [25030, 1961, 15167], 'ner_tags': ['O', 'B-PROBABILITY-AND-STATISTICS', 'I-PROBABILITY-AND-STATISTICS', 'I-PROBABILITY-AND-STATISTICS', 'O']}
differential forms:
 {'text_index': {0: 'differential', 1: 'forms'}, 'tokens': ['[CLS]', 'differential', 'forms', '[SEP]'], 'input_ids': [101, 12630, 2769, 102], 'ner_mapper': [12630, 2769], 'ner_tags': ['O', 'B-CALCULUS-AND-ANALYSIS', 'I-CALCULUS-AND-ANALYSIS', 'O']}


In [18]:
# Update tags to ignore exceptions such as prepositions and conjunctions
for key, sub_dict in all_classifications.items():
  input_ids = sub_dict['input_ids']
  ner_tags = sub_dict['ner_tags']
  for i in range(len(input_ids)):
    if input_ids[i] in exception_input_ids:
      ner_tags[i] = 'O'
  sub_dict['ner_tags'] = ner_tags

In [19]:
# Trim off input_ids 101 and 102 as these are the start and stop ids
input_id_mapping = dict()

for sub_dict in all_classifications.values():
  key = tuple(sub_dict['input_ids'][1:-1])
  value = sub_dict['ner_tags'][1:-1]
  assert len(key) == len(value), 'Error'
  input_id_mapping.update({key : value})

In [20]:
# Can also match on fuzzy-matched keys between the two eg "pythagoras's theorem" and "Pythagorean theorem"
dict_file_name = 'fuzzy_key_match_results.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    fuzz_dict = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")

  fuzz_dict = dict()
  all_classifications_keys = list(all_classifications.keys())

  for key in preliminary_ner_results.keys():
    my_dict = dict()
    for i in all_classifications_keys:
      fuzz_ratio = fuzz.ratio(key, i)
      if fuzz_ratio > 80 and fuzz_ratio < 100:
        my_dict.update({i : fuzz_ratio})
    fuzz_dict.update({key : my_dict})

  with open(dict_file_name, 'wb') as file:
    pickle.dump(fuzz_dict, file)

In [21]:
print(f"fuzzy_key_match_results dictionary length: {len(fuzz_dict)}")
random_sample = random.sample(list(fuzz_dict.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

fuzzy_key_match_results dictionary length: 5388
Discriminant:
 {'discriminant': 92}
Hasse Principle:
 {'hasse principle': 87}
Ambiguous Rectangle:
 {'ambiguous rectangle': 89}


In [22]:
# To increase the training dictionary size, get definitions for these fuzzy matches and append them to existing preliminary_ner_results dictionary
get_definition = []

for sub_dict in fuzz_dict.values():
  for key in sub_dict.keys():
    get_definition.append(key)

get_definition = list(set(get_definition))

In [23]:
# Get the API key and set the model name
model = "gpt-4o-mini"
client = OpenAI(api_key=userdata.get('ChatGPT'))

# Function to return answers based on prompt
def gpt_answers(topic):
  completion = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": f"""I will ask you to provide a short summary of a topic, 2 or 3 sentences long. Output must not include mathematical notation or history.
                  Example:

                  Question:
                  Markov chain

                  Answer:
                  A Markov Chain is a process where the next step depends only on the current state, not the path taken to get there. It consists of a set of states and the probabilities of transitioning from one state to another. This makes it a simple way to model systems that evolve step by step with memory only of the present.

                  End of example. """}, # system message that provides context to the model
      {"role": "user", "content": f"""Provide a short summary of: {topic}"""}  # topic query
    ]
  )

  response = completion.choices[0].message.content
  return response

In [24]:
try:
  # Load existing responses if the file exists
  with open('fuzzy_gpt_responses.pkl', 'rb') as file:
    fuzzy_gpt_responses = pickle.load(file)
except (FileNotFoundError, EOFError):
  fuzzy_gpt_responses = dict()

# Iterate over definitions and save each response as it's created
for definition in get_definition:
  if definition not in fuzzy_gpt_responses:
    fuzzy_gpt_responses[definition] = gpt_answers(definition)
    with open('fuzzy_gpt_responses.pkl', 'wb') as file:
      pickle.dump(fuzzy_gpt_responses, file)

In [25]:
print(f"fuzzy_gpt_responses dictionary length: {len(fuzzy_gpt_responses)}")
random_sample = random.sample(list(fuzzy_gpt_responses.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

fuzzy_gpt_responses dictionary length: 7512
cayley-hamilton theorem:
 The Cayley-Hamilton theorem states that every square matrix satisfies its own characteristic polynomial. This means that if you take a matrix, compute its characteristic polynomial, and then substitute the matrix itself into that polynomial, the result will yield the zero matrix. This theorem is a key concept in linear algebra and has applications in various areas such as control theory and engineering.
figurate numbers:
 Figurate numbers are numbers that can be represented as geometric shapes, such as triangles, squares, or pentagons. Each type of figurate number corresponds to a specific shape and is generated by a formula that relates to the number of points that can form that shape. They provide an interesting way to explore relationships between numbers and geometry.
neyman-pearson lemma:
 The Neyman-Pearson lemma is a fundamental result in statistical hypothesis testing that provides a method for determining th

In [26]:
# Encode new definitions
dict_file_name = 'ner_fuzzy_encoded.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    ner_fuzzy_encoded = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")
  start_time = time.time()

  ner_fuzzy_encoded = dict()

  for i, (key, value) in enumerate(fuzzy_gpt_responses.items()):
    if key not in ner_fuzzy_encoded:
      text_index = idx2string(value)
      # tokens = tag_tokenizer(value).tokens()
      tokenized = tag_tokenizer(value, padding=True, truncation=True)
      tokens = tokenized['input_ids']
      ner_results = tags(value)
      ner_fuzzy_encoded[key] = {
          "text" : value,
          "text_index": text_index,
          "text_start_end_index": idx2wordpos(value, text_index),
          "tokens" : tokens,
          "baseline_tags" : baseline_tags_list(tokens, ner_results),
          "input_ids" : tag_tokenizer.encode(value),
          "attention_mask" : tokenized['attention_mask']
          }

    if i % 1000 == 0 or i == len(fuzzy_gpt_responses) - 1:
      print(f"Processed {i+1} definitions, saving to file...")
      end_time = time.time()
      elapsed_time = (end_time - start_time) / 60
      print(f"***Execution time: {elapsed_time:.2f} minutes")
      with open(dict_file_name, 'wb') as file:
        pickle.dump(ner_fuzzy_encoded, file)

In [27]:
print(f"ner_fuzzy_encoded dictionary length: {len(ner_fuzzy_encoded)}")
random_sample = random.sample(list(ner_fuzzy_encoded.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

ner_fuzzy_encoded dictionary length: 7512
Niven's Theorem:
 {'text': "Niven's Theorem states that for any positive integer n, if n is the product of two integers, then at least one of these integers must divide n itself. The theorem focuses on the relationships between integers and their divisibility, providing insight into the nature of multiplication and factors. This concept is useful in number theory and can simplify various problems involving integer partitions and divisors.", 'text_index': {0: 'Niven', 1: "'", 2: 's', 3: 'Theorem', 4: 'states', 5: 'that', 6: 'for', 7: 'any', 8: 'positive', 9: 'integer', 10: 'n', 11: ',', 12: 'if', 13: 'n', 14: 'is', 15: 'the', 16: 'product', 17: 'of', 18: 'two', 19: 'integers', 20: ',', 21: 'then', 22: 'at', 23: 'least', 24: 'one', 25: 'of', 26: 'these', 27: 'integers', 28: 'must', 29: 'divide', 30: 'n', 31: 'itself', 32: '.', 33: 'The', 34: 'theorem', 35: 'focuses', 36: 'on', 37: 'the', 38: 'relationships', 39: 'between', 40: 'integers', 41: 'an

In [28]:
# Merge all data into single dictionary
all_data = dict()

for key_1, value_1 in ner_fuzzy_encoded.items():
  all_data.update({key_1 : value_1})

for key_2, value_2 in preliminary_ner_results.items(): #preliminary_ner_results
  if key_2 not in all_data:
    all_data.update({key_2 : value_2})

In [29]:
print(f"all_data dictionary length: {len(all_data)}")
random_sample = random.sample(list(all_data.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_data dictionary length: 11130
schur matrix:
 {'text': 'A Schur matrix is a special type of upper triangular matrix where each of its diagonal entries is an eigenvalue of the matrix. It is commonly used in numerical linear algebra, particularly in methods for solving eigenvalue problems. The Schur decomposition allows for simplifying the analysis of linear transformations and can be useful in various applications like control theory and signal processing.', 'text_index': {0: 'A', 1: 'Schur', 2: 'matrix', 3: 'is', 4: 'a', 5: 'special', 6: 'type', 7: 'of', 8: 'upper', 9: 'triangular', 10: 'matrix', 11: 'where', 12: 'each', 13: 'of', 14: 'its', 15: 'diagonal', 16: 'entries', 17: 'is', 18: 'an', 19: 'eigenvalue', 20: 'of', 21: 'the', 22: 'matrix', 23: '.', 24: 'It', 25: 'is', 26: 'commonly', 27: 'used', 28: 'in', 29: 'numerical', 30: 'linear', 31: 'algebra', 32: ',', 33: 'particularly', 34: 'in', 35: 'methods', 36: 'for', 37: 'solving', 38: 'eigenvalue', 39: 'problems', 40: '.', 41: 'Th

In [30]:
def find_sublist_indices(test_list, sub_list):
  n, m = len(test_list), len(sub_list)
  indices = []
  for i in range(n - m + 1):
    if test_list[i:i + m] == sub_list:
      indices.append(list(range(i, i + m)))
  return indices if indices else None  # Return None if no match is found

In [31]:
dict_file_name = 'all_data_matches.pkl'
list_file_name = 'non_matches.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    all_data = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")
  start_time = time.time()

  non_matches = set()

  for i, (key, sub_dict) in enumerate(all_data.items()):
    all_data_input_ids = sub_dict['input_ids']
    updated_tags = sub_dict['baseline_tags'].copy()
    # Only keep existing B, I-LOC and B, I-ORG tags
    updated_tags = ['O' if tag not in ['B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'] else tag for tag in updated_tags]
    my_list = []

    for classification_tuple, classification_ner in input_id_mapping.items():
      to_find = list(classification_tuple)
      idx_result = find_sublist_indices(all_data_input_ids, to_find)

      if idx_result:
        idx_list = [i for item in idx_result for i in item]
        my_list.append(idx_list)
        if len(idx_list) > len(classification_ner):
          classification_ner_list = list(len(idx_list)*classification_ner)
        if len(idx_list) == len(classification_ner):
          classification_ner_list = classification_ner
        if len(idx_list) < len(classification_ner):
          print(f"Error")
        for idx, classification in zip(idx_list, classification_ner):
          updated_tags[idx] = classification
      else:
        non_matches.add(key)
    ner_indices = [i for item in my_list for i in item]
    all_data[key]['ner_tags'] = updated_tags
    all_data[key]['ner_indices'] = ner_indices

    if i % 1000 == 0 or i == len(all_data) - 1:
      print(f"Processed {i+1} of {len(all_data)}, saving to file...")
      end_time = time.time()
      elapsed_time = (end_time - start_time)/60
      print(f" ***Execution time: {elapsed_time:.2f} minutes")
      with open(dict_file_name, 'wb') as file:
        pickle.dump(all_data, file)
      with open(list_file_name, 'wb') as file:
        pickle.dump(non_matches, file)

In [32]:
print(f"all_data dictionary length: {len(all_data)}")
random_sample = random.sample(list(all_data.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_data dictionary length: 11130
Perfect Square Dissection:
 {'text': 'Perfect square dissection involves dividing a perfect square into smaller squares without overlapping and covering the entirety of the original square. This geometric process creates a visually appealing arrangement, and is often used in mathematical puzzles and art. These dissections can reveal interesting properties about area and symmetry within a square framework.', 'text_index': {0: 'Perfect', 1: 'square', 2: 'dissection', 3: 'involves', 4: 'dividing', 5: 'a', 6: 'perfect', 7: 'square', 8: 'into', 9: 'smaller', 10: 'squares', 11: 'without', 12: 'overlapping', 13: 'and', 14: 'covering', 15: 'the', 16: 'entirety', 17: 'of', 18: 'the', 19: 'original', 20: 'square', 21: '.', 22: 'This', 23: 'geometric', 24: 'process', 25: 'creates', 26: 'a', 27: 'visually', 28: 'appealing', 29: 'arrangement', 30: ',', 31: 'and', 32: 'is', 33: 'often', 34: 'used', 35: 'in', 36: 'mathematical', 37: 'puzzles', 38: 'and', 39: 'art', 4

In [34]:
for_df = ['tokens', 'input_ids', 'baseline_tags', 'ner_tags', 'attention_mask']
my_list = []
for i in for_df:
  my_list.append(pd.DataFrame(all_data[random_sample[2][0]][i], columns=[i]))

df = pd.concat(my_list, axis=1)
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84
tokens,101,2464,1638,1132,16048,4184,1638,1115,3417,8803,170,3073,118,3597,2473,117,3423,117,1105,1510,170,1383,1104,2995,9042,1293,1103,1342,1110,1307,119,1220,1169,2079,1121,5564,1638,1115,4752,3607,2422,1106,6920,118,1359,1638,1115,12864,1113,2640,117,1543,1172,6736,1111,1672,6776,1105,4740,119,2464,1638,8343,1934,8234,117,1264,5361,117,1105,12176,4196,117,1510,2688,1112,170,1532,1104,5936,1105,8066,4506,119,102
input_ids,101,2464,1638,1132,16048,4184,1638,1115,3417,8803,170,3073,118,3597,2473,117,3423,117,1105,1510,170,1383,1104,2995,9042,1293,1103,1342,1110,1307,119,1220,1169,2079,1121,5564,1638,1115,4752,3607,2422,1106,6920,118,1359,1638,1115,12864,1113,2640,117,1543,1172,6736,1111,1672,6776,1105,4740,119,2464,1638,8343,1934,8234,117,1264,5361,117,1105,12176,4196,117,1510,2688,1112,170,1532,1104,5936,1105,8066,4506,119,102
baseline_tags,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
ner_tags,O,B-RECREATIONAL-MATHEMATICS,B-RECREATIONAL-MATHEMATICS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-APPLIED-MATHEMATICS,O,O,O,O,O,B-CALCULUS-AND-ANALYSIS,O,B-APPLIED-MATHEMATICS,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
attention_mask,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
